解析MNIST、mindspore标准数据集加载

In [10]:
import mindspore
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from mindspore import nn, Tensor
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset
from mindspore import dtype as mstype

In [20]:
# 自定义的mindspore标准格式数据集
from mindspore.mindrecord import FileWriter
import struct
# 解析ubyte文件，读取图片和标签  ./datasets_/
def parse_ubyte(ubyte, gen=False):
    if ubyte == "train":
        img_file = r"./datasets_/mindspore_mnist/train/train-images-idx3-ubyte"
        lbl_file = r"./datasets_/mindspore_mnist/train/train-labels-idx1-ubyte"
        label_file = open("parse_mnist/train_labels.txt", 'w')
        img_dir = "parse_mnist/train_images"
        if gen:
            os.makedirs(img_dir, exist_ok=True)
    elif ubyte == "test":
        img_file = r"./datasets_/mindspore_mnist/test/t10k-images-idx3-ubyte"
        lbl_file = r"./datasets_/mindspore_mnist/test/t10k-labels-idx1-ubyte"
        label_file = open("parse_mnist/test_labels.txt", 'w') 
        img_dir = "parse_mnist/test_images"
        if gen:
            os.makedirs(img_dir, exist_ok=True)
    else:
        print("输入ubyte不对")
        return None
    

    # 读取图片文件
    with open(img_file, 'rb') as f:
        # 读取文件头
        magic_number = struct.unpack('>I', f.read(4))[0]  # Magic number
        assert magic_number == 2051, f"{magic_number} Image file magic number incorrect!"
        
        num_images = struct.unpack('>I', f.read(4))[0]  # 读取图片数量
        rows = struct.unpack('>I', f.read(4))[0]  # 读取图片的行数
        cols = struct.unpack('>I', f.read(4))[0]  # 读取图片的列数
        print(f"num_images={num_images}, rows={rows}, cols={cols}")
        
        # 确保读取的字节数量与图片数据匹配
        image_data = np.fromfile(f, dtype=np.uint8)
        expected_size = num_images * rows * cols
        assert image_data.size == expected_size, f"Expected {expected_size} bytes, but got {image_data.size} bytes."
        
        # 将数据reshape为(num_images, rows, cols)
        images = image_data.reshape(num_images, rows, cols)

    # 读取标签文件
    with open(lbl_file, 'rb') as f:
        # 读取文件头
        magic_number = struct.unpack('>I', f.read(4))[0]  # Magic number
        assert magic_number == 2049, f"{magic_number} Label file magic number incorrect!"
        
        num_labels = struct.unpack('>I', f.read(4))[0]  # 读取标签数量
        labels_data = np.fromfile(f, dtype=np.uint8)  # 读取标签数据
        assert len(labels_data) == num_labels, f"Expected {num_labels} labels, but got {len(labels_data)} labels."
        
        labels = labels_data  # 保存标签数据

    # 如果gen=True，生成图片和标签文件
    if gen:
        for i in range(num_images):
            img = Image.fromarray(images[i])  # 将图片数据转换为PIL图片对象
            img.save(os.path.join(img_dir, f"{i+1}.jpg"))  # 保存图片为jpg文件
            label_file.write(f"{i+1}.jpg {labels[i]}\n")  # 写入标签文件

    label_file.close()
    # print(images.size, labels.size)
    return images, labels

# ## use
# parse_ubyte('train', gen=False)
# parse_ubyte('test', gen=True)


# def check_magic_number(file_path):
#     with open(file_path, 'rb') as f:
#         # 读取文件前4个字节（魔术数字应该在这4个字节中）
#         # magic_number = np.fromfile(f, dtype=np.uint32, count=1)[0]
#         magic_number = np.fromfile(f, dtype=np.dtype('>u4'), count=1)
#         print(f"Magic number: {magic_number}")
# train_img = r"./datasets_/mindspore_mnist/train/train-images-idx3-ubyte"
# # check_magic_number(train_img)


num_images=60000, rows=28, cols=28
47040000 60000


(array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],

In [15]:
# !python -m pip install tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [17]:
import tqdm
# 获取mindrecord格式数据集
def get_my_msrecord(mode, target_root= "parse_mnist/msrecord"):
    os.makedirs(target_root, exist_ok=True)
    
    if mode == "train":
        # 创建 MindRecord 文件路径
        mindrecord_path = os.path.join(target_root, 'train.mindrecord')
    elif mode == "test":
        # 创建 MindRecord 文件路径
        mindrecord_path = os.path.join(target_root, 'test.mindrecord')
    else:
        print("error input")
        return 
    
    writers = FileWriter(mindrecord_path, 32)  # 32 is the batch size
    
    # 构建字段
    schema = {
        "file_name": {"type": "string"},
        "label": {"type": "int32"},
        "data": {"type": "bytes"}}
    
    # 设置文件格式
    if mode == "train":
        writers.add_schema(schema, "train_schema")
        train_imgs, train_lbls = parse_ubyte(mode, False)
        print(train_imgs.size, train_lbls.size)
        # 写入训练集数据
        for i in tqdm.tqdm(range(len(train_imgs))):
            sample = {
                "file_name": f"train_{i}.jpg",
                "label": int(train_lbls[i]),
                "data": train_imgs[i].tobytes()  # 将图片数据转换为字节格式
            }
            writers.write_raw_data([sample])
    else:
        writers.add_schema(schema, "test_schema")
        test_imgs, test_lbls = parse_ubyte(mode, False)
        # 写入测试集数据
        for i in tqdm.tqdm(range(len(test_imgs))):
            sample = {
                "file_name": f"test_{i}.jpg",
                "label": int(test_lbls[i]),
                "data": test_imgs[i].tobytes()
            }
            writers.write_raw_data([sample])
    
    # 关闭文件
    writers.commit()

    print(f"MindRecord files created at {target_root}")

# get_my_msrecord("train")
# get_my_msrecord("test")

num_images=60000, rows=28, cols=28
47040000 60000


100%|██████████| 60000/60000 [00:00<00:00, 1838008.17it/s]


RuntimeError: 

In [53]:
from mindspore.dataset import MindDataset
import io

def file_test(file_list):
    for f in file_list:
        if not os.path.isfile(f):
            print(f)
            break


# 读取并打印数据集中的样本信息
def load_mindrecord_dataset(target_root, mode, batch_size=32):
    # 构造文件路径：train 或 test 数据集
    if mode == "train":
        mindrecord_files = [os.path.join(target_root,file) for file in os.listdir(target_root) if file.startswith('train.') and not file.endswith('.db')]  # 使用通配符读取多个文件
        # print(mindrecord_files)
    elif mode == "test":
        mindrecord_files = [os.path.join(target_root,file) for file in os.listdir(target_root) if file.startswith('test.') and not file.endswith('.db')]
    else:
        print("Error: Invalid mode!")
        return

    file_test(mindrecord_files)  # pass 
    
    # 加载 MindRecord 数据集，自动处理多个分片文件
    dataset = MindDataset(mindrecord_files)

    # 设置批次大小
    dataset = dataset.batch(batch_size)
    
    # 打印数据集的一些基本信息（如字段名和数据形状）
    for data in dataset.create_dict_iterator():
        print(f"Data sample:")
        for key, value in data.items():
            # print(f"  {key} shape: {value.shape if hasattr(value, 'shape') else 'No shape'}")
            # data: (32, 1)  file_name: (32,)  label: (32,)
            if key == "data":
                # print(f"  {key} sample: {value.size}")
                image_StubTensor = data['data']  # mindspore将字节码解析成StubTensor
                numpy_imgs = image_StubTensor.asnumpy()  
                # numpy_imgs.shape=(32, 1)  dtype='|S784'
                decoded_images = np.array(
                    [np.frombuffer(img, dtype=np.uint8).reshape(28, 28) 
                    for img in numpy_imgs])
                # decoded_images.shape=(32, 28, 28)  # b h w
                for i in range(decoded_images.shape[0]):
                    np_image = decoded_images[i]   
                    # print(f"{np_image.shape=}")  # np_image.shape=(28, 28)
                    label = data['label'][i].asnumpy()   # 获取标签（数字形式）
                    file_name = data['file_name'][i]
                    image = Image.fromarray(np_image) 
                    # print(f"{image.size=}")  # image.size=(28, 28)
                    image.convert("L").show()
                    break
                
            
        
        break  # 只打印第一个样本的结构，避免输出过多内容

    return dataset


# 示例：读取数据集并打印样本信息
mindrecord_path = r".\parse_mnist\msrecord"  # 替换为你的文件路径
load_mindrecord_dataset(mindrecord_path, mode='train')



Data sample:
  data: (32, 1)
  file_name: (32,)
  label: (32,)


In [47]:
msrecord = r"parse_mnist\msrecord"
def get_datsets(msrecord_root):
    train_mindrecord_files = [os.path.join(msrecord_root,file) 
                              for file in os.listdir(msrecord_root) 
                              if file.startswith('train.') and not file.endswith('.db')]  

    test_mindrecord_files = [os.path.join(msrecord_root,file) 
                             for file in os.listdir(msrecord_root) 
                             if file.startswith('test.') and not file.endswith('.db')]

    train_dataset = MindDataset(train_mindrecord_files, shuffle=True).batch(32)
    test_dataset = MindDataset(test_mindrecord_files, shuffle=False).batch(32)
    return train_dataset, test_dataset

train_ds, test_ds = get_datsets(msrecord)
for data in train_ds:
    print(data)
    break

[Tensor(shape=[32, 1], dtype=String, value=
[[b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00f\xe5\xff\xdb\xd7a\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xf9\xef\xd8\xe5\xf4\xfd\x9d\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00i\xfb\xee<\x00\x15,T\

In [57]:
import mindspore
from mindspore import nn, Tensor, Model
from mindspore.train.callback import LossMonitor
from mindspore.dataset import MindDataset
from mindspore import context
import os
import numpy as np
from PIL import Image

# 设置运行环境为图模式
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")  # 可以改为 "GPU"

# 加载数据集
msrecord = r"parse_mnist\msrecord"

def get_my_datasets(msrecord_root):
    # 构造文件路径
    train_mindrecord_files = [os.path.join(msrecord_root, file) 
                              for file in os.listdir(msrecord_root) 
                              if file.startswith('train.') and not file.endswith('.db')]  

    test_mindrecord_files = [os.path.join(msrecord_root, file) 
                             for file in os.listdir(msrecord_root) 
                             if file.startswith('test.') and not file.endswith('.db')]

    # 读取数据集并确保返回图像和标签
    def map_func(data):
        image_StubTensor = data['data']
        image = image_StubTensor.asnumpy()
        
        # 解码每个图像为 (28, 28) 的矩阵
        decoded_images = np.array([np.frombuffer(img, dtype=np.uint8).reshape(28, 28) for img in image])
        
        label = data['label'].asnumpy()  # 获取标签
        
        return Tensor(decoded_images), Tensor(label)

    
train_ds_, test_ds_ = get_datsets(msrecord)

# 示例：定义简单的卷积神经网络
class SimpleCNN(nn.Cell):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, pad_mode='same')
        self.conv2 = nn.Conv2d(32, 64, 3, pad_mode='same')
        self.flatten = nn.Flatten()
        self.fc = nn.Dense(64 * 28 * 28, num_classes)  # 28*28 是 MNIST 图像的大小

    def construct(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# 实例化网络模型
model = SimpleCNN(num_classes=10)

# 定义损失函数和优化器
loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

# 创建训练模型
train_model = Model(model, loss_fn=loss_fn, optimizer=optimizer, metrics={"accuracy"})

# 设置回调函数，每隔一定步骤输出训练损失
loss_monitor = LossMonitor(per_print_times=100)

# 开始训练
train_model.train(10, train_ds_, callbacks=[loss_monitor], dataset_sink_mode=False)


RuntimeError: Exception thrown from dataset pipeline. Refer to 'Dataset Pipeline Error Message'. 

------------------------------------------------------------------
- Dataset Pipeline Error Message: 
------------------------------------------------------------------
[ERROR] Invalid file, failed to open files for reading mindrecord files. Please check file path, permission and open files limit(ulimit -a): parse_mnist\msrecord\train.mindrecord31.

------------------------------------------------------------------
- C++ Call Stack: (For framework developers) 
------------------------------------------------------------------
mindspore\ccsrc\minddata\mindrecord\io\shard_reader.cc(239).


